In [145]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from scipy import stats
import math

In [146]:
ps = pd.read_csv('patch_seq_log_mouse.csv')

In [1]:
#ps.head()

In [147]:
ps = ps[["Date", "User", "Post patch?", "Post patch pipette R", "Nucleus sucked in?", "Notes","Internal version", "PCR cycles", "SM_QC_PF", "Resolution index", "Bad dates"]]
#ps.dropna(subset = ["SM_QC_PF", "Post patch?"], inplace = True)
ps.dropna(subset = ["Post patch?"], inplace = True)
ps["User"] = ps["User"].astype("category")
ps["Post patch?"] = ps["Post patch?"].astype("category")
ps["SM_QC_PF"] = ps["SM_QC_PF"].astype("category")
ps = ps[ps["Bad dates"] != 'x']
ps = ps[ps["SM_QC_PF"] != "cnt"]
ps = ps[ps["PCR cycles"] == 21]
ps = ps.drop('Bad dates', 1)
ps["Date"] = pd.to_datetime(ps["Date"], format = '%y%m%d')
#ps["PCR cycles"].value_counts()
ps

,Date,User,Post patch?,Post patch pipette R,Nucleus sucked in?,Notes,Internal version,PCR cycles,SM_QC_PF,Resolution index
4717,2017-04-19,P2,Outside-Out,NaN,NaN,NaN,AiV2,21.0,fail,NaN
4718,2017-04-19,P2,Outside-Out,NaN,NaN,NaN,AiV2,21.0,pass,0.923406
4719,2017-04-19,P2,No-Seal,NaN,NaN,NaN,AiV2,21.0,fail,NaN
4720,2017-04-19,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,0.000000
4721,2017-04-19,P2,Outside-Out,NaN,NaN,NaN,AiV2,21.0,pass,0.322631
4722,2017-04-19,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,0.322631
4723,2017-04-19,P2,No-Seal,NaN,NaN,NaN,AiV2,21.0,pass,0.000000
4724,2017-04-19,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,NaN
4727,2017-04-19,P6,No-Seal,NaN,NaN,NaN,AiV2,21.0,fail,NaN
4728,2017-04-19,P6,No-Seal,NaN,NaN,NaN,AiV2,21.0,pass,0.000000


In [148]:
#total = ps["Post patch?"].count()
total = get_count(ps)
total
#ps['Post patch?'].value_counts()

1726

#### defining filtering functions

In [149]:
def nucleated(x):
    nucleus = x[(x["Post patch?"] == "Nucleated") | (((x["Post patch?"] == "nucleus_visible") | (x["Post patch?"] == "nucleus_present")) & (x["Post patch pipette R"] >= 500))]
    return nucleus

def partial_nucleated(y):
    partial = y[(y["Post patch?"] == 'Partial-Nucleus') | (((y['Post patch?'] == 'nucleus_present') | (y['Post patch?'] == 'nucleus_visible')) & (y["Post patch pipette R"] <= 500))]
    return partial



#### defining calculating functions

In [150]:
def get_count(x):    
    count = x["Post patch?"].count()
    return count

def calc_pp(passing,total):
    pp = float(get_count(passing)) / get_count(total)
    return pp

#### Filtering to nucleated patches only and calculating percent nucleated

In [151]:
#nuc = ps[(ps["Post patch?"] == "Nucleated") | (ps["Post patch?"] == "Partial-Nucleus") | (ps["Post patch?"] == "nucleus_present") | (ps["Post patch?"] == "nucleus_visible")]
#nuc = ps[(ps["Post patch?"] == "Nucleated") | (((ps["Post patch?"] == "nucleus_visible") | (ps["Post patch?"] == "nucleus_present")) & (ps["Post patch pipette R"] >= 500))]
nuc = nucleated(ps)
nuc.tail()
#dic = {'Nucleated': 'test',
    #  'nucelated': 'Nucleated'
    # }

#ps['Post patch?'] = ps['Post patch?'].map(lambda x: dic[x])

,Date,User,Post patch?,Post patch pipette R,Nucleus sucked in?,Notes,Internal version,PCR cycles,SM_QC_PF,Resolution index
7197,2017-10-05,P2,nucleus_present,1000.0,no,Fluorescence in Pipette No Bubbles alexa fill ...,AiV2-50uM alexa 488,21.0,NaN,NaN
7200,2017-10-05,P8,nucleus_present,1200.0,no,"Fluorescence in Pipette,Cell Dimmed,Cell Swell...",AiV2-50uM alexa 488,21.0,NaN,NaN
7202,2017-10-05,P9,nucleus_present,1800.0,no,"Cell Dimmed Small Bubbles,Solution in Pipette ...",AiV2-50uM alexa 488,21.0,NaN,NaN
7205,2017-10-05,P9,nucleus_present,980.0,no,Cell Dimmed No Bubbles,AiV2-50uM alexa 488,21.0,NaN,NaN
7207,2017-10-05,PA,nucleus_present,2000.0,no,Cell Shrunk No Bubbles,AiV2-50uM alexa 488,21.0,NaN,NaN


In [152]:
#nuc_count = nuc["Post patch?"].count()
nuc_count = get_count(nuc)
nuc_count

806

In [153]:
#nuc_percent = float(nuc_count)/total
#nuc_percent
nuc_pct = calc_pp(nuc, ps) 
print "nucleated patch %:", nuc_pct

nucleated patch %: 0.46697566628


#### Filtering to partial-nucleated patches only and calculating percent partial-nucleated

In [154]:
part_nuc = partial_nucleated(ps)
part_nuc

,Date,User,Post patch?,Post patch pipette R,Nucleus sucked in?,Notes,Internal version,PCR cycles,SM_QC_PF,Resolution index
4745,2017-04-20,P1,Partial-Nucleus,NaN,NaN,NaN,AiV2,21.0,pass,0.678447
4770,2017-04-21,P1,Partial-Nucleus,NaN,NaN,"Fluorescence in Pipette,Cell Dimmed",AiV2,21.0,pass,1.000000
4796,2017-04-24,P1,Partial-Nucleus,NaN,NaN,Cell Dimmed lots of small bubbles,AiV2,21.0,fail,NaN
4884,2017-04-27,P1,Partial-Nucleus,NaN,NaN,Fluorescence in Pipette lots of small bubbles,AiV2,21.0,fail,NaN
4918,2017-04-28,P2,Partial-Nucleus,NaN,NaN,one tiny bubble formed during expulsion.,AiV2,21.0,pass,1.000000
4920,2017-04-28,P2,Partial-Nucleus,NaN,NaN,no bubbles during expulsion.,AiV2,21.0,pass,1.000000
4937,2017-05-01,P2,Partial-Nucleus,NaN,NaN,Fluorescence in Pipette One big bubble formed ...,AiV2,21.0,pass,NaN
4939,2017-05-01,P2,Partial-Nucleus,NaN,NaN,Fluorescence in Pipette,AiV2,21.0,pass,NaN
4951,2017-05-01,P6,Partial-Nucleus,NaN,NaN,NaN,AiV2,21.0,fail,NaN
4952,2017-05-01,P6,Partial-Nucleus,NaN,NaN,NaN,AiV2,21.0,fail,NaN


In [155]:
part_nuc_pct = calc_pp(part_nuc, ps)
print "partial nucleated patch %:", part_nuc_pct

partial nucleated patch %: 0.125144843569


# Nucleated patch percentage without P1

In [70]:
no_p1_total = ps[ps["User"] != 'P1']
no_p1_total

,Date,User,Post patch?,Post patch pipette R,Nucleus sucked in?,Notes,Internal version,PCR cycles,SM_QC_PF,Resolution index
4717,2017-04-19,P2,Outside-Out,NaN,NaN,NaN,AiV2,21.0,fail,NaN
4718,2017-04-19,P2,Outside-Out,NaN,NaN,NaN,AiV2,21.0,pass,0.923406
4719,2017-04-19,P2,No-Seal,NaN,NaN,NaN,AiV2,21.0,fail,NaN
4720,2017-04-19,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,0.000000
4721,2017-04-19,P2,Outside-Out,NaN,NaN,NaN,AiV2,21.0,pass,0.322631
4722,2017-04-19,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,0.322631
4723,2017-04-19,P2,No-Seal,NaN,NaN,NaN,AiV2,21.0,pass,0.000000
4724,2017-04-19,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,NaN
4727,2017-04-19,P6,No-Seal,NaN,NaN,NaN,AiV2,21.0,fail,NaN
4728,2017-04-19,P6,No-Seal,NaN,NaN,NaN,AiV2,21.0,pass,0.000000


In [71]:
no_p1_nuc = no_p1_total[no_p1_total["Post patch?"] == "Nucleated"]
no_p1_nuc

,Date,User,Post patch?,Post patch pipette R,Nucleus sucked in?,Notes,Internal version,PCR cycles,SM_QC_PF,Resolution index
4720,2017-04-19,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,0.000000
4722,2017-04-19,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,0.322631
4724,2017-04-19,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,NaN
4758,2017-04-20,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,0.678447
4775,2017-04-21,P2,Nucleated,NaN,NaN,Fluorescence in Pipette,AiV2,21.0,pass,1.000000
4776,2017-04-21,P2,Nucleated,NaN,NaN,Fluorescence in Pipette,AiV2,21.0,pass,0.877682
4784,2017-04-21,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,0.556320
4807,2017-04-24,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,0.678447
4812,2017-04-24,P2,Nucleated,NaN,NaN,Fluorescence in Pipette One big bubble formed ...,AiV2,21.0,pass,0.678447
4833,2017-04-25,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,1.000000


In [72]:
no_p1_count = no_p1_total["Post patch?"].count()
no_p1_count

1180

In [73]:
no_p1_nuccount = no_p1_nuc["Post patch?"].count()
no_p1_nuccount

566

In [74]:
new_no_p1_percent = float(no_p1_nuccount)/no_p1_count
new_no_p1_percent

0.47966101694915253

# Calculating nucleated patch percentages by user

In [75]:
P8_total = ps[ps["User"] == 'P8']

P8_total_count = P8_total['Post patch?'].count()
P8_total_count

419

In [76]:
P8_nuc = P8_total[P8_total['Post patch?'] == 'Nucleated']
P8_nuc_count = P8_nuc['Post patch?'].count()
P8_nuc_count

211

In [77]:
float(P8_nuc_count)/P8_total_count

0.5035799522673031

In [78]:
for riguser in ['P1','P2','P6','P8']:
    user_total = ps[ps['User'] == riguser]
    user_total_count = user_total['Post patch?'].count()
    user_nuc = user_total[user_total['Post patch?'] == 'Nucleated']
    user_nuc_count = user_nuc['Post patch?'].count()
    user_percent = float(user_nuc_count)/(user_total_count)
    print riguser,': ', user_percent
    

P1 :  0.342490842491
P2 :  0.493554327808
P6 :  0.369863013699
P8 :  0.503579952267


# Obtaining nucleus dependant on internal solution version

In [156]:
standard_internal = ps[ps["Internal version"] == "AiV2"]
standard_internal = standard_internal[(standard_internal["User"] == "P2") | (standard_internal["User"] == "P8")]
standard_internal.head()
                       

,Date,User,Post patch?,Post patch pipette R,Nucleus sucked in?,Notes,Internal version,PCR cycles,SM_QC_PF,Resolution index
4717,2017-04-19,P2,Outside-Out,NaN,NaN,NaN,AiV2,21.0,fail,NaN
4718,2017-04-19,P2,Outside-Out,NaN,NaN,NaN,AiV2,21.0,pass,0.923406
4719,2017-04-19,P2,No-Seal,NaN,NaN,NaN,AiV2,21.0,fail,NaN
4720,2017-04-19,P2,Nucleated,NaN,NaN,NaN,AiV2,21.0,pass,0.000000
4721,2017-04-19,P2,Outside-Out,NaN,NaN,NaN,AiV2,21.0,pass,0.322631


In [157]:
#total_standard_internal = standard_internal["Post patch?"].count()
total_standard_internal = get_count(standard_internal)
total_standard_internal

759

In [160]:
alexa_internal = ps[ps["Internal version"] == "AiV2-50uM alexa 488"]
alexa_internal = alexa_internal[(alexa_internal["User"] == "P2") | (alexa_internal["User"] == "P8")]
alexa_internal

,Date,User,Post patch?,Post patch pipette R,Nucleus sucked in?,Notes,Internal version,PCR cycles,SM_QC_PF,Resolution index
6843,2017-08-29,P8,Entire-Cell,NaN,NaN,"Fluorescence in Pipette,Cell Dimmed,Cell Shrun...",AiV2-50uM alexa 488,21.0,pass,0.322631
6844,2017-08-29,P8,Partial-Nucleus,NaN,NaN,"Fluorescence in Pipette,Cell Dimmed,Cell Swell...",AiV2-50uM alexa 488,21.0,pass,0.000000
6845,2017-08-30,P8,Partial-Nucleus,NaN,NaN,"Fluorescence in Pipette,Cell Swelled No Bubble...",AiV2-50uM alexa 488,21.0,pass,0.783192
6848,2017-08-30,P8,No-Seal,NaN,NaN,"Fluorescence in Pipette,Cell Shrunk No Bubbles...",AiV2-50uM alexa 488,21.0,pass,0.000000
6850,2017-08-30,P8,No-Seal,NaN,NaN,Fluorescence in Pipette No Bubbles,AiV2-50uM alexa 488,21.0,pass,0.322631
6857,2017-08-31,P2,Nucleated,NaN,NaN,No Bubbles 0.1 units of pressure on expulsion,AiV2-50uM alexa 488,21.0,pass,1.000000
6858,2017-08-31,P2,Nucleated,NaN,NaN,Fluorescence in Pipette No Bubbles 0.1 units o...,AiV2-50uM alexa 488,21.0,pass,1.000000
6859,2017-08-31,P2,Nucleated,NaN,NaN,Fluorescence in Pipette No Bubbles 0.1 units o...,AiV2-50uM alexa 488,21.0,pass,1.000000
6862,2017-08-31,P8,Nucleated,NaN,NaN,"Fluorescence in Pipette,Cell Dimmed,Cell Shrun...",AiV2-50uM alexa 488,21.0,pass,1.000000
6863,2017-08-31,P8,No-Seal,NaN,NaN,Cell Disappeared,AiV2-50uM alexa 488,21.0,pass,0.469928


In [162]:
#total_alexa_internal = alexa_internal["Post patch?"].count()
total_alexa_internal = get_count(alexa_internal)
total_alexa_internal

203

In [163]:
#standard_internal_nuc = standard_internal[(standard_internal["Post patch?"] == "Nucleated") | (standard_internal["Post patch?"] == "Partial-Nucleus") | (standard_internal["Post patch?"] == "nucleus_present")|(standard_internal["Post patch?"] == "nucleus_visible")]
standard_internal_nuc = nucleated(standard_internal)
standard_internal_nuc
#standard_internal_nuc_count = standard_internal_nuc["Post patch?"].count()
standard_internal_nuc_pp = calc_pp(standard_internal_nuc, standard_internal)
standard_internal_nuc_pp
#print "standard internal:" , float(standard_internal_nuc_count)/total_standard_internal

0.5125164690382081

In [164]:
#alexa_internal_nuc = alexa_internal[(alexa_internal["Post patch?"] == "Nucleated") | (alexa_internal["Post patch?"] == "Partial-Nucleus") | (alexa_internal["Post patch?"] == "nucleus_present")|(alexa_internal["Post patch?"] == "nucleus_visible")]
alexa_internal_nuc = nucleated(alexa_internal)
alexa_internal_nuc
#alexa_internal_nuc_count = alexa_internal_nuc["Post patch?"].count()
alexa_internal_nuc_pp = calc_pp(alexa_internal_nuc, alexa_internal)
alexa_internal_nuc_pp
#print "alexa internal:" , float(alexa_internal_nuc_count)/total_alexa_internal

0.5812807881773399

# Rate of SMQC pass-fail by internal solution verion

In [165]:
standard_internal.dropna(subset = ["SM_QC_PF", "Post patch?"], inplace = True)
standard_internal_count = standard_internal["Post patch?"].count()
standard_internal_count

758

In [166]:
standard_internal_pass = standard_internal[standard_internal["SM_QC_PF"] == "pass"]
standard_internal_pass_count = standard_internal_pass["Post patch?"].count()
standard_internal_pass_count                                    

661

In [167]:
print "Standard internal pass rate:" , float(standard_internal_pass_count)/standard_internal_count

Standard internal pass rate: 0.872031662269


In [168]:
alexa_internal.dropna(subset = ["SM_QC_PF", "Post patch?"], inplace = True)
alexa_internal_count = alexa_internal["Post patch?"].count()
alexa_internal_count

83

In [89]:
alexa_internal_pass = alexa_internal[alexa_internal["SM_QC_PF"] == "pass"]
alexa_internal_pass_count = alexa_internal_pass["Post patch?"].count()
alexa_internal_pass_count

71

In [90]:
print "Alexa internal pass rate:" , float(alexa_internal_pass_count)/alexa_internal_count

Alexa internal pass rate: 0.855421686747


## Quick test to figure out how to filter based on multiple conlums at once

In [91]:
#ps.tail(20)

In [92]:
part_nuc = ps[(ps["Post patch?"] == 'Partial-Nucleus') | (((ps['Post patch?'] == 'nucleus_present') | (ps['Post patch?'] == 'nucleus_visible')) & (ps["Post patch pipette R"] <= 500))]
part_nuc

,Date,User,Post patch?,Post patch pipette R,Nucleus sucked in?,Notes,Internal version,PCR cycles,SM_QC_PF,Resolution index
4745,2017-04-20,P1,Partial-Nucleus,NaN,NaN,NaN,AiV2,21.0,pass,0.678447
4770,2017-04-21,P1,Partial-Nucleus,NaN,NaN,"Fluorescence in Pipette,Cell Dimmed",AiV2,21.0,pass,1.000000
4796,2017-04-24,P1,Partial-Nucleus,NaN,NaN,Cell Dimmed lots of small bubbles,AiV2,21.0,fail,NaN
4884,2017-04-27,P1,Partial-Nucleus,NaN,NaN,Fluorescence in Pipette lots of small bubbles,AiV2,21.0,fail,NaN
4918,2017-04-28,P2,Partial-Nucleus,NaN,NaN,one tiny bubble formed during expulsion.,AiV2,21.0,pass,1.000000
4920,2017-04-28,P2,Partial-Nucleus,NaN,NaN,no bubbles during expulsion.,AiV2,21.0,pass,1.000000
4937,2017-05-01,P2,Partial-Nucleus,NaN,NaN,Fluorescence in Pipette One big bubble formed ...,AiV2,21.0,pass,NaN
4939,2017-05-01,P2,Partial-Nucleus,NaN,NaN,Fluorescence in Pipette,AiV2,21.0,pass,NaN
4951,2017-05-01,P6,Partial-Nucleus,NaN,NaN,NaN,AiV2,21.0,fail,NaN
4952,2017-05-01,P6,Partial-Nucleus,NaN,NaN,NaN,AiV2,21.0,fail,NaN
